In [106]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr


%config InlineBackend.figure_format = 'png' #set 'png' here when working on notebook
%matplotlib inline



In [107]:
train = pd.read_csv("./train.csv")

#iteration1
train.drop('Condition2',axis=1,inplace=True)
train.drop('Street',axis=1,inplace=True)
train.drop('Alley',axis=1,inplace=True)
train.drop('LandContour',axis=1,inplace=True)
train.drop('Utilities',axis=1,inplace=True)
train.drop('LandSlope',axis=1,inplace=True)
train.drop('RoofStyle',axis=1,inplace=True)
train.drop('ExterCond',axis=1,inplace=True)
train.drop('BsmtFinType2',axis=1,inplace=True)
train.drop('Electrical',axis=1,inplace=True)
train.drop('GarageQual',axis=1,inplace=True)
train.drop('PavedDrive',axis=1,inplace=True)
train.drop('PoolQC',axis=1,inplace=True)
train.drop('Fence',axis=1,inplace=True)



test = pd.read_csv("./test.csv")

In [108]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LotConfig,Neighborhood,Condition1,BldgType,...,3SsnPorch,ScreenPorch,PoolArea,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Reg,Inside,CollgCr,Norm,1Fam,...,0,0,0,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Reg,FR2,Veenker,Feedr,1Fam,...,0,0,0,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,IR1,Inside,CollgCr,Norm,1Fam,...,0,0,0,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,IR1,Corner,Crawfor,Norm,1Fam,...,0,0,0,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,IR1,FR2,NoRidge,Norm,1Fam,...,0,0,0,NaN,0,12,2008,WD,Normal,250000


In [109]:
all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))

In [110]:
#log transform the target:
train["SalePrice"] = np.log1p(train["SalePrice"])

#log transform skewed numeric features:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

In [111]:
all_data = pd.get_dummies(all_data)


In [112]:
#filling NA's with the mean of the column:
all_data = all_data.fillna(all_data.mean())

In [113]:
#creating matrices for sklearn:
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train.SalePrice

In [114]:
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

In [115]:
def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

In [116]:
model_lasso = LassoCV(alphas = [1, 0.1, 0.001, 0.0005],max_iter=1000).fit(X_train, y)

In [122]:
rmse_cv(model_lasso).mean()

0.121026996189413

In [118]:
lasso_preds = np.expm1(model_lasso.predict(X_test))


In [119]:


preds = lasso_preds



In [121]:
solution = pd.DataFrame({"id":test.Id, "SalePrice":preds})
solution.to_csv("lasso.csv", index = False)
